In [4]:
import pandas  as pd

In [5]:
df = pd.read_csv('final_df_to_train.csv')
agr = df.groupby('themes').count()
ind = agr[agr['Автор'] > 20].index

In [6]:
data = df[df.themes.isin(ind)]

In [17]:
data[['Обращение полное', 'Сообщение ЧАВО', 'themes']].head()

,Обращение полное,Сообщение ЧАВО,themes
0,"здравствуйте, в приложении не сохраняется мой ...","Обновите, пожалуйста, приложение до последней ...",105;1053;nan;
1,"Добрый день, вы снова повысили лимит для перех...","Нет, таких АЗС нет.",102;None;nan;
2,"Обратная связь по обновленном приложению ,Рань...",Благодарим Вас за пользование приложением сети...,101;1011;101108.0;
3,"Здравствуйте, хочу активировать ещё одну карту...",В соответствии с условиями Программы лояльност...,101;1012;nan;
4,"Сколько карт можно добавить в приложение?,При...",В соответствии с условиями Программы лояльност...,101;1012;nan;


In [34]:
data.themes.unique().shape

(18,)

##  Модель

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import joblib 

/home/varykha/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [23]:
#SVC after grid_search (final)
le = LabelEncoder()
le.fit(data['themes'])

X = data['Обращение полное']
#Y = le.transform(data['themes'])
Y = data['themes']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state=241)
vectorizer_n = TfidfVectorizer()

count_vect = CountVectorizer()
X_train = count_vect.fit_transform(X_train)

clf_svc = SVC(gamma='scale', decision_function_shape='ovr', C = 100, random_state=241, kernel='rbf')
clf_svc.fit(X_train, y_train)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=241, shrinking=True, tol=0.001,
    verbose=False)

In [24]:
X_test = count_vect.transform(X_test)
pred = clf_svc.predict(X_test)

In [25]:
accuracy_score(y_test, pred)

0.7844896331738437

In [51]:
#SVC
clf_svc = SVC()
grid_svc = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf']}
kf_svc = sklearn.model_selection.KFold(shuffle=True, random_state=1, n_splits=5)
gs_svc = GridSearchCV(clf_svc, grid_svc, scoring='accuracy')
gs_svc.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['linear', 'rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [3]:
print('best score: ', gs_svc.best_score_)
print('best estimatorr: ', gs_svc.best_estimator_)

NameError: name 'gs_svc' is not defined

In [43]:
X1 = count_vect.transform(['Не заправиться через приложение'])
clf_svc.predict(X1)

array(['105;1051;nan;'], dtype=object)

In [36]:
r = data[['Обращение полное', 'Сообщение ЧАВО']][data['themes'] == "105;1051;nan;"]

In [39]:
r.iloc[0, 0]

'Вечер добрый\nНахожусь на АЗС 201,пытался заправится через приложение,но оно не работает.Не позволяет выбрать количество литров/рублей.Уже не первый раз.Может настроил что не так?Прошу помочь в данном вопросе'

In [42]:
r.iloc[0, 1]

'Мобильное приложение работает по системе «предоплата». Чтобы заправиться с помощью мобильного приложения АЗС GО: 1.\tОстановитесь у свободной топливно-раздаточной колонки 2.\tВ приложении выберите АЗС, на которой находитесь. 3.\tУкажите номер колонки, наименование топлива и объем 4.\tУкажите эл. почту для получения чека об оплате 5.\tОплатите заказ 6.\tДождитесь завершения обработки заказа 7.\tНе снимайте пистолет до указания помощника в приложении 8.\tПосле одобрения помощником вставьте пистолет в бак автомобиля 9.\tДождитесь окончания заправки и верните пистолет в бак'

In [48]:
joblib.dump(clf_svc, 'svc.pkl') 

['svc.pkl']

In [49]:
joblib.dump(count_vect, 'count_vect.pkl') 

['count_vect.pkl']

In [50]:
pickle.dump(clf_svc, open('svc.pkl', 'wb'))
pickle.dump(count_vect, open('count_vect.pkl', 'wb'))